In [ ]:
import os

def get_method_name(log_path: str) -> str:
    file_name = os.path.splitext(os.path.basename(log_path))[0]
    file_name_split = file_name.split('_')
    
    if file_name_split[4] == 'EDF':
        method_name = 'EDF'
    else:
        method_name = file_name_split[1]

    return method_name

In [ ]:
from typing import Dict

def get_classification(log: Dict) -> str:
    if 'deadline_miss' in log.keys() and 'early_detection' in log.keys():
        classification = 'TP'
    elif 'deadline_miss' not in log.keys() and 'early_detection' in log.keys():
        classification = 'FP'
    elif 'deadline_miss' in log.keys() and 'early_detection' not in log.keys():
        classification = 'FN'
    elif 'deadline_miss' not in log.keys() and 'early_detection' not in log.keys():
        classification += 'TN'

    return classification

# Total utilization varied

In [ ]:
def get_utilization(log_path: str) -> float:
    file_name = os.path.splitext(os.path.basename(log_path))[0]
    utilization_str = file_name.split('_')[-1]
    utilization = float(utilization_str.split('=')[1])

    if 0.65 <= utilization < 0.7:
        utilization = 65
    elif 0.7 <= utilization < 0.75:
        utilization = 70
    elif 0.75 <= utilization < 0.8:
        utilization = 75
    elif 0.8 <= utilization < 0.85:
        utilization = 80
    elif 0.85 <= utilization < 0.9:
        utilization = 85
    elif 0.9 <= utilization < 0.95:
        utilization = 90
    elif 0.95 <= utilization < 1.0:
        utilization = 95

    return utilization

In [ ]:
import glob
import pandas as pd
import yaml

METHOD_NAMES = ['proposed', 'Igarashi', 'Saidi', 'EDF']

utilization_df_dict = {}
for method in METHOD_NAMES:
    utilization_df_dict[method] = pd.DataFrame(index=['TP Ratio', 'FN Ratio', 'TN Ratio', 'FP Ratio'],
                                               columns=['65', '70', '75', '80', '85', '90', '95'])
    utilization_df_dict[method].fillna(0, inplace=True)

In [ ]:
log_paths = glob.glob("./results/utilization_varied/*.yaml")

for log_path in log_paths:
    with open(log_path, "r") as f:
        log = yaml.safe_load(f)
    
    method_name = get_method_name(log_path)
    utilization = get_utilization(log_path)
    classification = get_classification(log)
    utilization_df_dict[method_name].at[f'{classification} Ratio', str(utilization)] += 1

In [ ]:
for method in METHOD_NAMES:
    utilization_df_dict[method] = \
        utilization_df_dict[method].div(utilization_df_dict[method].sum(axis=0), axis=1)

In [ ]:
utilization_df_dict['proposed']

In [ ]:
utilization_df_dict['Igarashi']

In [ ]:
utilization_df_dict['Saidi']

In [ ]:
utilization_df_dict['EDF']

# Alpha varied

In [ ]:
def get_alpha(log_path: str) -> float:
    file_name = os.path.splitext(os.path.basename(log_path))[0]
    alpha_str = file_name.split('_')[2]
    alpha = float(alpha_str.split('=')[1])

    return round(alpha, 1)

In [ ]:
import glob
import pandas as pd
import yaml

METHOD_NAMES = ['proposed', 'Igarashi', 'Saidi', 'EDF']

alpha_df_dict = {}
for method in METHOD_NAMES:
    alpha_df_dict[method] = pd.DataFrame(index=['TP Ratio', 'FN Ratio', 'TN Ratio', 'FP Ratio'],
                                         columns=[]) # TODO
    alpha_df_dict[method].fillna(0, inplace=True)

In [ ]:
log_paths = glob.glob("./results/alpha_varied/*.yaml")

for log_path in log_paths:
    with open(log_path, "r") as f:
        log = yaml.safe_load(f)
    
    method_name = get_method_name(log_path)
    alpha = get_alpha(log_path)
    classification = get_classification(log)
    alpha_df_dict[method_name].at[f'{classification} Ratio', str(alpha)] += 1

In [ ]:
for method in METHOD_NAMES:
    alpha_df_dict[method] = \
        alpha_df_dict[method].div(alpha_df_dict[method].sum(axis=0), axis=1)

In [ ]:
alpha_df_dict['proposed']

In [ ]:
alpha_df_dict['Igarashi']

In [ ]:
alpha_df_dict['Saidi']

In [ ]:
alpha_df_dict['EDF']

# Earlier time

In [ ]:
earlier_time_dict = {'proposed': [],
                     'Igarashi': [],
                     'Saidi': [],
                     'EDF': []}

log_paths = glob.glob("./results/**/*.yaml", recursive=True)
for log_path in log_paths:
    with open(log_path, "r") as f:
        log = yaml.safe_load(f)

    if get_classification(log) == 'TP':
        earlier_time = (log['deadline_miss']['deadline_miss_time']
                        - log['early_detection']['detection_time'])
        earlier_time_dict[get_method_name(log)].append(earlier_time)

In [ ]:
METHOD_NAMES = ['proposed', 'Igarashi', 'Saidi', 'EDF']

for method in METHOD_NAMES:
    print(f'{method} ave: {sum(earlier_time_dict[method])/len(earlier_time_dict[method])}')
    print(f'{method} max: {max(earlier_time_dict[method])}')